<a href="https://colab.research.google.com/github/jeremysb1/xgboost/blob/main/xgboost_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

I am building a model to predict prices using a regression-based approach with XGBoost.

In [4]:
import pandas as pd
summary_listings = pd.read_csv("/content/drive/MyDrive/XGBoost/listings.csv")

In [5]:
summary_listings.dtypes

id                                  int64
name                               object
host_id                             int64
host_name                          object
neighbourhood_group               float64
neighbourhood                      object
latitude                          float64
longitude                         float64
room_type                          object
price                               int64
minimum_nights                      int64
number_of_reviews                   int64
last_review                        object
reviews_per_month                 float64
calculated_host_listings_count      int64
availability_365                    int64
number_of_reviews_ltm               int64
license                            object
dtype: object

In [6]:
summary_listings['name'].iloc[0]

'Rental unit in Sumida · ★4.77 · 1 bedroom · 2 beds · 1 bath'

Extracting information from text descriptions:

In [7]:
import re

classification_list = ['aparthotel', 'barn', 'bed and breakfast', 'boutique hotel',
                       'bungalow', 'cabin', 'camper/rv', 'chalet', 'condo', 'cottage',
                       'earthen home', 'farm stay', 'guest suite', 'guesthouse', 'home',
                       'hostel', 'hotel', 'houseboat', 'hut', 'loft', 'place to stay',
                       'rental unit', 'resort', 'ryokan', 'serviced apartment',
                       'tiny home', 'townhouse', 'treehouse', 'vacation home', 'villa']

summary_listings = summary_listings.assign(
    type_of_accommodation=(summary_listings['name']
                           .str.extract(f"({'|'.join(classification_list)})",
                                        flags=re.IGNORECASE)),
    area_of_tokyo=(summary_listings['name']
                   .str.extract(r'in\s(.*?)\s·',
                                flags=re.IGNORECASE)),
    score=(summary_listings['name']
           .str.extract(r'★(\d+\.\d+)', flags=re.IGNORECASE)
           .astype(float)),
    number_of_bedrooms=(summary_listings['name']
                        .str.extract(r'(\d+)\s*(?:bedroom|bedrooms)',
                                     flags=re.IGNORECASE)
                        .fillna(0)
                        .astype(int)),
    number_of_beds=(summary_listings['name']
                    .str.extract(r'(\d+)\s+(?:beds?\b)',
                                 flags=re.IGNORECASE)
                    .fillna(0)
                    .astype(int)),
    number_of_baths=(summary_listings['name']
                     .str.extract(r'(?P<baths>\d+)\s*(shared\s+)?(?:half-)?baths?\b',
                                  flags=re.IGNORECASE)["baths"]
                     .fillna(0)
                     .astype(int)),
)

In [ ]:
import numpy as np
from datetime import datetime

